In [1]:
import torch
import numpy as np

In [8]:
pred_labels = torch.from_numpy(np.random.choice([0, 1], size=(2, 500, 128)))
n_models, n_samples, seq_len = pred_labels.size()
pred_labels_flattened = pred_labels.view((n_models, n_samples*seq_len))
agg_labels = []
model_id = 0
for single_examples_pred_labels in pred_labels_flattened.T:
    labels, counts = torch.unique(single_examples_pred_labels, return_counts=True)
    agg_labels.append(labels[torch.argmax(counts)])
torch.tensor(agg_labels).view(n_samples, seq_len).shape

torch.Size([500, 128])

In [23]:
pred_probs = torch.zeros(size=(16, 128, 2), dtype=pred_labels.dtype)

a, b = torch.argwhere(pred_labels).T
pred_probs[a, b, 1] = 1

a, b = torch.argwhere(pred_labels == 0).T
pred_probs[a, b, 0] = 1
pred_probs

tensor([[[1, 0],
         [1, 0],
         [0, 1],
         ...,
         [0, 1],
         [0, 1],
         [1, 0]],

        [[1, 0],
         [0, 1],
         [1, 0],
         ...,
         [0, 1],
         [1, 0],
         [0, 1]],

        [[0, 1],
         [1, 0],
         [0, 1],
         ...,
         [1, 0],
         [1, 0],
         [0, 1]],

        ...,

        [[0, 1],
         [1, 0],
         [1, 0],
         ...,
         [1, 0],
         [0, 1],
         [1, 0]],

        [[0, 1],
         [0, 1],
         [0, 1],
         ...,
         [0, 1],
         [0, 1],
         [1, 0]],

        [[0, 1],
         [1, 0],
         [0, 1],
         ...,
         [0, 1],
         [0, 1],
         [1, 0]]], dtype=torch.int32)

In [37]:
logits = np.random.random(size=(16, 128))

np.apply_along_axis(lambda x: x in np.array([0, 1]), axis=0, arr=np.unique(logits).flatten())

C:\Users\holaj\AppData\Local\Temp\ipykernel_29960\1050868711.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  np.apply_along_axis(lambda x: x in np.array([0, 1]), axis=0, arr=np.unique(logits).flatten())


array(False)

In [7]:
contain_bool = np.vectorize(lambda x: x in np.array([0, 1]))
contain_bool(np.unique(logits).flatten()).all()

NameError: name 'logits' is not defined